# IBM Ponder This - February 2024

## Problem Statement

Alice and Bob play the following weird dice game using a standard set of 6 D&D dice, i.e., dice with values

$1,\ldots,4$ <br>
$1,\ldots,6$ <br>
$1,\ldots,8$ <br>
$0,\ldots,9$ <br>
$1,\ldots,12$ <br>
$1,\ldots,20$ 

At each round, they throw the dice and sum their values. All six dice need to be rolled once each round, and it doesn’t matter who rolls the dice. What matters in deciding who wins is the sum.


Alice wins if the result is a prime number, and Bob wins if the result is a nonprime even number. Otherwise, the result of the round is a draw.


The winner of the game is the first player to reach N consecutive wins.


Your goal: For $N=13$ , find the probability that Alice wins the game.

A Bonus "*" will be given for finding the probability that Alice wins given that $N=300$ but the rules have changed and Bob now wins if the result is a nonprime odd number.

## Solution

First we generate the distribution of the sum of dices (frequency of occurence of each sum) and compute the winning probability of each player and the probability of a draw. The winning probability of Alice, $p_A$, is the sum of frequencies of prime numbers divided by the total sum of frequencies of all sums. The probability of winning for Bob, $p_B$, is the sum of frequencies of even nonprime sums divided by the total sum of frequencies of all sums. The probability of drawing is $p_D = 1 - p_A - p_B$.

Then we use a markov chain to determine the probability that Alice wins 13 consecutive games. Let $S_{0}$ denote the probability from the initial state at the start of the game or after a draw. Let $S_i$ with $1 \leq i \leq 13$ denote the states where Alice has $i$ consecutive wins and $S_{i}$ with $-13 \leq i \leq -1$ denote the states where Bob has $i$ consecutive wins.

When in $S_0$, either Alice wins with probability $p_A$ and we move to $S_1$ or Bob wins with probability $p_B$ and we move to $S_{-1}$ or there is a draw and we stay in state $S_0$. From any state $1 \leq i < 13$, we can either go to the next positive state with probability $p_A$, go to $S_{-1}$ with probability $p_B$ or go back to $S_0$ with probability $p_D$. Similarly, from any state $-13 < i \leq -1$, we either move to the next negative state with probability $p_B$, move to state $S_1$ with probability $p_A$ or go back to $S_0$ with probability $p_D$. Finally we have the two final states, $S_{13} = 1$ when Alice wins and $S_{-13} = 0$ when Bob wins. We can describe all the states with the following system of equations

\begin{align}
    &S_0 = p_A S_1 + p_B S_{-1} + p_D S_0 \\
    &S_i = \begin{cases} p_A S_{i + 1} + p_B S_{-1} + p_D S_0, \quad 1 \leq i < 13 \\
    p_A S_1+ p_B S_{i - 1} + p_D S_0, \quad -13 < i \leq  -1
    \end{cases} \\
    &S_{13} = 1 \\
    &S_{-13} = 0
\end{align}

Now we simply need to solve the system of equations for $S_0$ to obtain the answer.

In [1]:
import sympy
from sympy import symbols, Eq, solve
from collections import defaultdict

In [2]:
# Compute the distribution
d = defaultdict(int)
for i in range(1, 5):
    for j in range(1, 7):
        for k in range(1, 9):
            for l in range(10):
                for m in range(1, 13):
                    for n in range(1, 21):
                        d[i + j + k + l + m + n] += 1

# Determine the probability of winning for each player and probability of a draw
alice = 0
bob = 0
draw = 0
for key, val in d.items():
    if sympy.isprime(key):
        alice += val
    elif key % 2 == 0:
        bob += val
    else:
        draw += val
p_alice = sympy.simplify(f"{alice} / ({alice} + {bob} + {draw})")
p_bob = sympy.simplify(f"{bob} / ({alice} + {bob} + {draw})")
p_draw = sympy.simplify(f"{draw} / ({alice} + {bob} + {draw})")

print(f'Probability that Alice wins is {p_alice}')
print(f'Probability that Bob wins is {p_bob}')
print(f'Probability of a draw is {p_draw}')

Probability that Alice wins is 12711/51200
Probability that Bob wins is 1/2
Probability of a draw is 12889/51200


In [3]:
# Create and solve the system of equations

P, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12, A13, B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11, B12, B13, p, q, r = symbols('P A1 A2 A3 A4 A5 A6 A7 A8 A9 A10 A11 A12 A13 B1 B2 B3 B4 B5 B6 B7 B8 B9 B10 B11 B12 B13 p q r')

eq0 = Eq(P, A1*p + B1*q + P*r)
eq1 = Eq(A1, A2*p + B1*q + P*r)
eq2 = Eq(A2, A3*p + B1*q + P*r)
eq3 = Eq(A3, A4*p + B1*q + P*r)
eq4 = Eq(A4, A5*p + B1*q + P*r)
eq5 = Eq(A5, A6*p + B1*q + P*r)
eq6 = Eq(A6, A7*p + B1*q + P*r)
eq7 = Eq(A7, A8*p + B1*q + P*r)
eq8 = Eq(A8, A9*p + B1*q + P*r)
eq9 = Eq(A9, A10*p + B1*q + P*r)
eq10 = Eq(A10, A11*p + B1*q + P*r)
eq11 = Eq(A11, A12*p + B1*q + P*r)
eq12 = Eq(A12, A13*p + B1*q + P*r)
eq13 = Eq(A13, 1)

eq21 = Eq(B1, A1*p + B2*q + P*r)
eq22 = Eq(B2, A1*p + B3*q + P*r)
eq23 = Eq(B3, A1*p + B4*q + P*r)
eq24 = Eq(B4, A1*p + B5*q + P*r)
eq25 = Eq(B5, A1*p + B6*q + P*r)
eq26 = Eq(B6, A1*p + B7*q + P*r)
eq27 = Eq(B7, A1*p + B8*q + P*r)
eq28 = Eq(B8, A1*p + B9*q + P*r)
eq29 = Eq(B9, A1*p + B10*q + P*r)
eq30 = Eq(B10, A1*p + B11*q + P*r)
eq31 = Eq(B11, A1*p + B12*q + P*r)
eq32 = Eq(B12, A1*p + B13*q + P*r)
eq33 = Eq(B13, 0)

solution = solve((eq0, eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9, eq10, eq11, eq12, eq13, eq21, eq22, eq23, eq24, eq25, eq26, eq27, eq28, eq29, eq30, eq31, eq32, eq33), (P, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12, A13, B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11, B12, B13))

sympy.simplify(solution[P])

p**13*(-q**12 - q**11 - q**10 - q**9 - q**8 - q**7 - q**6 - q**5 - q**4 - q**3 - q**2 - q - 1)/(p**12*q**12*r + p**12*q**12 + p**12*q**11*r + p**12*q**11 + p**12*q**10*r + p**12*q**10 + p**12*q**9*r + p**12*q**9 + p**12*q**8*r + p**12*q**8 + p**12*q**7*r + p**12*q**7 + p**12*q**6*r + p**12*q**6 + p**12*q**5*r + p**12*q**5 + p**12*q**4*r + p**12*q**4 + p**12*q**3*r + p**12*q**3 + p**12*q**2*r + p**12*q**2 + p**12*q*r + p**12*q + p**12*r + p**11*q**12*r + p**11*q**12 + p**11*q**11*r + p**11*q**11 + p**11*q**10*r + p**11*q**10 + p**11*q**9*r + p**11*q**9 + p**11*q**8*r + p**11*q**8 + p**11*q**7*r + p**11*q**7 + p**11*q**6*r + p**11*q**6 + p**11*q**5*r + p**11*q**5 + p**11*q**4*r + p**11*q**4 + p**11*q**3*r + p**11*q**3 + p**11*q**2*r + p**11*q**2 + p**11*q*r + p**11*q + p**11*r + p**10*q**12*r + p**10*q**12 + p**10*q**11*r + p**10*q**11 + p**10*q**10*r + p**10*q**10 + p**10*q**9*r + p**10*q**9 + p**10*q**8*r + p**10*q**8 + p**10*q**7*r + p**10*q**7 + p**10*q**6*r + p**10*q**6 + p**10*q**5

In [4]:
# Get the result
res = sympy.simplify(solution[P]).subs(p, p_alice).subs(q, p_bob).subs(r, p_draw)
sympy.simplify(res)

1852137689209162765594102138215850956181263861797578044921/11053138293799686957338442574894173280443057797092491883862521

In [5]:
res.evalf(15)

0.000167566680157085

In [6]:
# Avoid symbolic computation

p = p_alice
q = p_bob
r = p_draw

P, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12, A13, B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11, B12, B13 = symbols('P A1 A2 A3 A4 A5 A6 A7 A8 A9 A10 A11 A12 A13 B1 B2 B3 B4 B5 B6 B7 B8 B9 B10 B11 B12 B13')

eq0 = Eq(P, A1*p + B1*q + P*r)
eq1 = Eq(A1, A2*p + B1*q + P*r)
eq2 = Eq(A2, A3*p + B1*q + P*r)
eq3 = Eq(A3, A4*p + B1*q + P*r)
eq4 = Eq(A4, A5*p + B1*q + P*r)
eq5 = Eq(A5, A6*p + B1*q + P*r)
eq6 = Eq(A6, A7*p + B1*q + P*r)
eq7 = Eq(A7, A8*p + B1*q + P*r)
eq8 = Eq(A8, A9*p + B1*q + P*r)
eq9 = Eq(A9, A10*p + B1*q + P*r)
eq10 = Eq(A10, A11*p + B1*q + P*r)
eq11 = Eq(A11, A12*p + B1*q + P*r)
eq12 = Eq(A12, A13*p + B1*q + P*r)
eq13 = Eq(A13, 1)

eq21 = Eq(B1, A1*p + B2*q + P*r)
eq22 = Eq(B2, A1*p + B3*q + P*r)
eq23 = Eq(B3, A1*p + B4*q + P*r)
eq24 = Eq(B4, A1*p + B5*q + P*r)
eq25 = Eq(B5, A1*p + B6*q + P*r)
eq26 = Eq(B6, A1*p + B7*q + P*r)
eq27 = Eq(B7, A1*p + B8*q + P*r)
eq28 = Eq(B8, A1*p + B9*q + P*r)
eq29 = Eq(B9, A1*p + B10*q + P*r)
eq30 = Eq(B10, A1*p + B11*q + P*r)
eq31 = Eq(B11, A1*p + B12*q + P*r)
eq32 = Eq(B12, A1*p + B13*q + P*r)
eq33 = Eq(B13, 0)

solution = solve((eq0, eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8, eq9, eq10, eq11, eq12, eq13, eq21, eq22, eq23, eq24, eq25, eq26, eq27, eq28, eq29, eq30, eq31, eq32, eq33), (P, A1, A2, A3, A4, A5, A6, A7, A8, A9, A10, A11, A12, A13, B1, B2, B3, B4, B5, B6, B7, B8, B9, B10, B11, B12, B13))

sympy.simplify(solution[P])

1852137689209162765594102138215850956181263861797578044921/11053138293799686957338442574894173280443057797092491883862521

In [7]:
# Generate equations in a cleaner way for given N

def generate_equations(N):
    P = sympy.symbols('P')
    A = sympy.symbols(f'A1:A{N+1}')
    B = sympy.symbols(f'B1:B{N+1}')

    # Initial equation for P
    equations = [sympy.Eq(P, A[0]*p_alice + B[0]*p_bob + P*p_draw)]

    # Equations for A_i
    for i in range(N):
        if i < N-1:
            equations.append(sympy.Eq(A[i], A[i+1]*p_alice + B[0]*p_bob + P*p_draw))
        else:
            equations.append(sympy.Eq(A[i], 1))  # Final state for A_N
    # Equations for B_i
    for i in range(N):
        if i < N-1:
            equations.append(sympy.Eq(B[i], A[0]*p_alice + B[i+1]*p_bob + P*p_draw))
        else:
            equations.append(sympy.Eq(B[i], 0))  # Final state for B_N

    # Solve the equations
    solution = sympy.solve(equations, (P, *A, *B))
    return sympy.simplify(solution[P])

# Example usage for N = 13
P_value_for_N13 = generate_equations(13)
P_value_for_N13


1852137689209162765594102138215850956181263861797578044921/11053138293799686957338442574894173280443057797092491883862521